In [1]:
import numpy as np
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

import datetime as datetime

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

import kernels

In [2]:
%load_ext autoreload
%autoreload 2
import solution
from solution import Model
from solution import cv_eval


In [ ]:
#copy code to override default classes

In [3]:
#def main():
train_x_name = "train_x.csv"
train_y_name = "train_y.csv"

train_x = np.loadtxt(train_x_name, delimiter=',')
train_y = np.loadtxt(train_y_name, delimiter=',')

# load the test dateset
test_x_name = "test_x.csv"
test_x = np.loadtxt(test_x_name, delimiter=',')

In [4]:
df_vals = np.stack([train_x[:,0],train_x[:,1],train_y],axis=1)

df = pd.DataFrame(data = df_vals,columns = ['x0','x1','y'])

df_left = df[df['x0']<-0.5]
df_left = df_left.sample(frac=0.1,random_state=42)

train_x = df_left[['x0','x1']].values
train_y = df_left['y'].values

print(df_left.shape)

(1710, 3)


In [5]:
def tune_HP_nested_CV(kernels_grid, K_cv, model_config):
    train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
 
    CV_res = []
    for hp_kernel in kernels_grid:
        model_config["kernel"] = hp_kernel
        cv_m = cv_eval(cv_splits = K_cv,
               cv_preprocess_left_frac = 0.05,
               model_config=model_config)
        val_cost_array = cv_m.run_cross_validation(train_x_split, train_y_split)
        print(np.mean(val_cost_array))
        print(np.std(val_cost_array))
        CV_res.append(np.mean(val_cost_array))
    return CV_res

print(np.shape(train_x))
#nested n1*k*n2 total
def tune_HP_nested_CV2(kernels_grid, K_cv, model_config,result_csv_path):
    n_outer = 5
    n_kernels = len(kernels_grid)
    n_inner = K_cv
    cv_config = {"outer":n_outer,"kernels":n_kernels,"inner":n_inner}
    
    cv_outer = KFold(n_splits=n_outer, shuffle=True, random_state=42)
    outer_res = []
    inner_res = np.zeros((n_outer,n_kernels))

    outer_count = 0
    for train_ix,test_ix in cv_outer.split(train_x):
        train_x_split,test_x_split = train_x[train_ix,:],train_x[test_ix,:]
        train_y_split,test_y_split = train_y[train_ix],train_y[test_ix]
        #train_x_split, test_x_split, train_y_split, test_y_split = train_test_split(train_x, train_y, test_size=0.33, random_state=42)
 
        kernel_count = 0
        for hp_kernel in kernels_grid:
            cv_result_cols = ["test_cost","val_cost","val_lml","init_kernel",
                              "left_data","cv_config","model_config","time"]
            df_cv_result = pd.DataFrame(columns = cv_result_cols)
            #df_cv_result = pd.read_csv(result_csv_path)
        
            print("outer=%d"%outer_count)
            print("kernel="+str(hp_kernel))
            model_config["kernel"] = hp_kernel
            cv_m = cv_eval(cv_splits = K_cv,
                   cv_preprocess_left_frac = 1.0,
                   model_config=model_config)
            val_cost_array,lml_array = cv_m.run_cross_validation(train_x_split, train_y_split)

            #inner_res = np.append(inner_res,np.mean(val_cost_array))
            inner_res[outer_count][kernel_count] = np.mean(val_cost_array)
            
            test_model = solution.Model(model_config)
            test_model.fit_model(train_x_split,train_y_split)
            test_y_pred = test_model.predict(test_x_split)
            test_cost = solution.cost_function(test_y_split,test_y_pred)
            
            print(test_cost)
            print(np.mean(val_cost_array))
            print(np.std(val_cost_array))
            
            df_cv_result = df_cv_result.append({
                          "test_cost":test_cost,
                          "val_cost":np.mean(val_cost_array),
                          "val_lml":np.mean(lml_array),
                          "init_kernel":str(hp_kernel),
                          "left_data":int(df_left.shape[0]),
                          "cv_config":str(cv_config),
                          "model_config":str(model_config),
                          "time":datetime.datetime.now()
                         },ignore_index=True)
            df_cv_result.to_csv(result_csv_path,index=False,header=False,mode="a")
            kernel_count+=1            
        outer_count += 1

    return {"outer":outer_res,"inner":inner_res}

(1710, 2)


 # Call Scikit-learn 

In [ ]:
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern, RBF

#kernels_grid = [kernels.sklearn_best()]
kernels_grid = [
    RBF(),
    1.0*RBF()+WhiteKernel(),
    1.0*Matern(nu=1.5)+WhiteKernel(),
    1.0*Matern(nu=2.5)+WhiteKernel(),
    1.0+1.0*Matern(nu=1.5)+WhiteKernel(),
    1.0+1.0*Matern(nu=2.5)+WhiteKernel()
]
K_cv = 3
model_config = {
    "use_skit_learn":True,
    "use_nystrom":False,
    "use_fitc" : False,
    "model_preprocess_left_frac":1.0
}

result_csv_path = "cv_results.csv"
tune_HP_nested_CV2(kernels_grid, K_cv, model_config,result_csv_path)

outer=0
kernel=RBF(length_scale=1)
(1368, 3)
i=0
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


i=1
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


i=2
logging_setup
training cost fn   0.051183
val cost fn        7.114097
val log likelihood fn        -319859718.017057


[-3.19859718e+08 -3.19859718e+08 -3.19859718e+08]
logging_setup
4.7014748839108
7.1140974392303065
0.0
outer=0
kernel=1**2 * RBF(length_scale=1) + WhiteKernel(noise_level=1)
(1368, 3)
i=0
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


i=1
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


i=2
logging_setup
training cost fn   0.054964
val cost fn        0.054739
val log likelihood fn        1282.128277


[1282.12827689 1282.12827689 1282.12827689]
logging

In [ ]:

# ### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
# kernel = kernels.sklearn_best()
# K_cv = 10
# model_config = {
#     "kernel":kernel,
#     "use_skit_learn":True,
#     "model_preprocess_left_frac":1.0
# }

# cv_m = cv_eval(cv_splits = K_cv,
#                cv_preprocess_left_frac = 0.05,
#                model_config=model_config) # Choose whether to use custom model or scikit learn
# val_cost_array = cv_m.run_cross_validation(train_x, train_y)

# print(np.mean(val_cost_array))
# print(np.std(val_cost_array))

# Call FITC

In [ ]:
kernels_grid = [kernels.sklearn_best()]
K_cv = 10
model_config = {
    "use_skit_learn":False,
    "use_nystrom":False,
    "use_fitc":True,
    "model_preprocess_left_frac":1.0
}
tune_HP_nested_CV(kernels_grid, K_cv, model_config)

In [ ]:
# ### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
# kernel = kernels.sklearn_best()
# K_cv = 10
# model_config = {
#     "kernel":kernel,
#     "use_skit_learn":False,
#     "use_nystrom":False,
#     "use_fitc":True,
#     "model_preprocess_left_frac":1.0
# }
# cv_m = cv_eval(cv_splits = K_cv,
#                cv_preprocess_left_frac = 0.05,
#                model_config=model_config) # Choose whether to use custom model or scikit learn
# val_cost_array = cv_m.run_cross_validation(train_x, train_y)

# print(np.mean(val_cost_array))
# print(np.std(val_cost_array))

In [ ]:
cv_m.models

# Call Nystrom

In [ ]:
kernels_grid = [kernels.sklearn_best()]
K_cv = 10
model_config = {
    "use_skit_learn":False,
    "use_nystrom":True,
    "use_fitc":False,
    "model_preprocess_left_frac":1.0
}
tune_HP_nested_CV(kernels_grid, K_cv, model_config)

In [ ]:
# ### TODO add a simple outer train, test split to select the hyperparameters (kernel type etc) in the inner CV loop 
# kernel = kernels.sklearn_best()
# K_cv = 10
# model_config = {
#     "kernel":kernel,
#     "use_skit_learn":False,
#     "use_nystrom":True,
#     "use_fitc":False,
#     "model_preprocess_left_frac":1.0
# }
# cv_m = cv_eval(cv_splits = K_cv,
#                cv_preprocess_left_frac = 0.05,
#                model_config=model_config) # Choose whether to use custom model or scikit learn
# val_cost_array = cv_m.run_cross_validation(train_x, train_y)

# print(np.mean(val_cost_array))
# print(np.std(val_cost_array))


# Create New CV Results csv

In [6]:
#Normally no need to call

cv_result_cols = ["test_cost","val_cost","lml","init_kernel","left_data","cv_config","model_config","time"]
df_cv_result = pd.DataFrame(columns = cv_result_cols)
#df_cv_result
result_csv_path = "cv_results.csv"
df_cv_result.to_csv(result_csv_path,index=False,mode="w")

# Temporary 

In [ ]:
#check nystrom

In [ ]:
train_x_trunc = train_x[:100]
train_x_full = train_x[:10000]

In [ ]:
kernel = kernels.sklearn_best()
print(kernel(train_x_trunc,train_x_trunc))


In [ ]:
model_config = {"use_skit_learn":False,
                "use_nystrom":True,
                "nystrom_q":100,
                "kernel":kernel,
                "variance":0.01,
                "correct_y_pred":False,
                "preprocess_frac":0.05}
model = Model(model_config)

In [ ]:
model.fit_model(train_x,train_y)
model.


In [ ]:
Knn = kernels.rbf_kernel(train_x_full,train_x_full)
Kqq = kernels.rbf_kernel(train_x_trunc,train_x_trunc)
Knq = kernels.rbf_kernel(train_x_full,train_x_trunc)
res = np.linalg.eig(Kqq)
Lambdaqq = np.diag(res[0])
Uqq = res[1]

In [ ]:
#print(np.sum(res[0]).imag)
print(np.absolute(res[0]))
